<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/rust01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

1. Colab で実行しながら読まれることを想定している。
1. Rust の勉強
1. 資料は本家のテュートリアル https://rust-cli.github.io/book/index.html

# Rust のインストール





In [ ]:
# Rust をインストールする。
%%capture
!apt install rustc --fix-missing

通常の curl を使ったインストールは使えない。

In [ ]:
# 参考

# !curl https://sh.rustup.rs -sSf | sh
#=> sh: 121: cannot open /dev/tty: No such device or address

In [8]:
# インストールの確認
!rustc --version

rustc 1.43.0


In [ ]:
# 環境をつくるコマンドの cargo も入っている
!cargo --version

cargo 1.43.0


In [ ]:
%env USER=root
!cargo init

env: USER=root
     Created binary (application) package


以上で、ここでmain.rsをビルドして、コンパイルすることができるようになる。

In [ ]:
# プログラムを書く
%%writefile main.rs
fn main() {
    println!("Hello, world!");
}

Writing main.rs


In [ ]:
# コンパイル
!cargo build

   Compiling content v0.1.0 (/content)
    Finished dev [unoptimized + debuginfo] target(s) in 0.40s


In [ ]:
# コンパイルして実行
!cargo run 

    Finished dev [unoptimized + debuginfo] target(s) in 0.01s
     Running `target/debug/content`
Hello, world!


# テュートリアルより



以上で Colab で Rust チュートリアルのノートをとる準備が整ったが、チュートリアル自体は Colab 用ではないので次のような記述になっている。

まず、`cargo new grrs` (grrsは今回作るプログラム名)、でディレクトリーを作る。 grrs の中にいろいろなファイルができる。 なかでも `Cargo.toml` にプロジェクトのメタデータが書かれていて、使われるライブラリーの依存、バージョンコントロールが行われる。 cargo run すると、main.rs がコンパイルされ、実行される。

In [ ]:
# ざっと次のような流れになる
%%script false
$ cargo new grrs
     Created binary (application) `grrs` package
$ cd grrs/
$ cargo run
   Compiling grrs v0.1.0 (/Users/pascal/code/grrs)
    Finished dev [unoptimized + debuginfo] target(s) in 0.70s
     Running `target/debug/grrs`
Hello, world!

セルマジックの %%script false が書かれるとそれ以降の行は実行されない。

全体をコメントアウトしたような使い方ができる。

## 最初の課題

In [ ]:
# テュートリアルより最初の課題
# 次のような動作をするgrepもどき(grrs)をつくる。

# $ cat test.txt
# foo: 10
# bar: 20
# baz: 30
# $ grrs foo test.txt
# foo: 10
# $ grrs --help
# [some help text explaining the available options]


## コマンドラインをパースする

# いまここ

典型的なコマンドラインツールはつぎのような形になる。


In [ ]:
%%script false
$ grrs foobar test.txt

この例では、grrs というプログラムで、test.txt の中の foobar という文字列を含む行をプリントアウトすることを想定している。

プログラム名のあとの文字列は大雑把にいうとスペースで区切られている文字列のリストとしてオペレーティングシステムの内部的には処理されている。

コマンドライン引数と呼ばれる。 --this というような書き方でフラグをしていするのにも使われる。




## コマンドライン引数の取得

標準ライブラリーに関数 
std::env::args() 
があり、戻り値はイテレーターで index 0 にはプログラム自身、今回の例では grrs が入っている。








Getting the arguments
The standard library contains the function that gives you an iterator of the given arguments. The first entry (at index 0) will be the name your program was called as (e.g. grrs), the ones that follow are what the user wrote afterwards.

Getting the raw arguments this way is quite easy (in file src/main.rs, after fn main() {):




let pattern = std::env::args().nth(1).expect("no pattern given");
let path = std::env::args().nth(2).expect("no path given");
CLI arguments as data type


In [ ]:

let pattern = std::env::args().nth(1).expect("no pattern given");
let path = std::env::args().nth(2).expect("no path given");
CLI arguments as data type


Instead of thinking about them as a bunch of text, it often pays off to think of CLI arguments as a custom data type that represents the inputs to your program.



Look at grrs foobar test.txt: There are two arguments, first the pattern (the string to look for), and then the path (the file to look in).

What more can we say about them? Well, for a start, both are required. We haven’t talked about any default values, so we expect our users to always provide two values. Furthermore, we can say a bit about their types: The pattern is expected to be a string, while the second argument is expected to be a path to a file.

In Rust, it is common to structure programs around the data they handle, so this way of looking at CLI arguments fits very well. Let’s start with this (in file src/main.rs, before fn main() {):




struct Cli {
    pattern: String,
    path: std::path::PathBuf,
}

This defines a new structure (a struct) that has two fields to store data in: pattern, and path.

Aside: PathBuf is like a String but for file system paths that works cross-platform.

Now, we still need to get the actual arguments our program got into this form. One option would be to manually parse the list of strings we get from the operating system and build the structure ourselves. It would look something like this:




let pattern = std::env::args().nth(1).expect("no pattern given");
let path = std::env::args().nth(2).expect("no path given");
let args = Cli {
    pattern: pattern,
    path: std::path::PathBuf::from(path),
};
This works, but it’s not very convenient. How would you deal with the requirement to support --pattern="foo" or --pattern "foo"? How would you implement --help?

Parsing CLI arguments with StructOpt
A much nicer way is to use one of the many available libraries. The most popular library for parsing command line arguments is called clap. It has all the functionality you’d expect, including support for sub-commands, shell completions, and great help messages.

The structopt library builds on clap and provides a “derive” macro to generate clap code for struct definitions. This is quite nice: All we have to do is annotate a struct and it’ll generate the code that parses the arguments into the fields.

Let’s first import structopt by adding structopt = "0.3.13" to the [dependencies] section of our Cargo.toml file.

Now, we can write use structopt::StructOpt; in our code, and add #[derive(StructOpt)] right above our struct Cli. Let’s also write some documentation comments along the way.

It’ll look like this (in file src/main.rs, before fn main() {):




use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

Note: There are a lot of custom attributes you can add to fields. For example, we added one to tell structopt how to parse the PathBuf type. To say you want to use this field for the argument after -o or --output, you’d add #[structopt(short = "o", long = "output")]. For more information, see the structopt documentation.

Right below the Cli struct our template contains its main function. When the program starts, it will call this function. The first line is:




fn main() {
    let args = Cli::from_args();
}
This will try to parse the arguments into our Cli struct.

But what if that fails? That’s the beauty of this approach: Clap knows which fields to expect, and what their expected format is. It can automatically generate a nice --help message, as well as give some great errors to suggest you pass --output when you wrote --putput.

Note: The from_args method is meant to be used in your main function. When it fails, it will print out an error or help message and immediately exit the program. Don’t use it in other places!

This is what it may look like
Running it without any arguments:




$ cargo run
    Finished dev [unoptimized + debuginfo] target(s) in 10.16s
     Running `target/debug/grrs`
error: The following required arguments were not provided:
    <pattern>
    <path>

USAGE:
    grrs <pattern> <path>

For more information try --help
We can pass arguments when using cargo run directly by writing them after --:


$ cargo run -- some-pattern some-file
    Finished dev [unoptimized + debuginfo] target(s) in 0.11s
     Running `target/debug/grrs some-pattern some-file`
As you can see, there is no output. Which is good: That just means there is no error and our program ended.

Exercise for the reader: Make this program output its arguments!

In [ ]:
%%writefile main.rs
fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

Overwriting main.rs


In [ ]:
# 実験
!cargo build

In [18]:
%%writefile Cargo.toml
[package]
name = "content"
version = "0.1.0"
authors = ["root"]
edition = "2018"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[dependencies]
structopt = "0.3.13"

[[bin]]
name = "content"
path = "main.rs"

Overwriting Cargo.toml


In [9]:
%%writefile main.rs
use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

Overwriting main.rs


In [12]:
!cat main.rs

use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

In [15]:
!cargo update

In [19]:
!cargo build

    Updating crates.io index
  Downloaded structopt v0.3.20
  Downloaded clap v2.33.3
  Downloaded structopt-derive v0.4.13
  Downloaded lazy_static v1.4.0
  Downloaded textwrap v0.11.0
  Downloaded strsim v0.8.0
  Downloaded atty v0.2.14
  Downloaded vec_map v0.8.2
  Downloaded libc v0.2.80
  Downloaded syn v1.0.48
  Downloaded quote v1.0.7
  Downloaded unicode-width v0.1.8
  Downloaded ansi_term v0.11.0
  Downloaded bitflags v1.2.1
  Downloaded proc-macro2 v1.0.24
  Downloaded heck v0.3.1
  Downloaded proc-macro-error v1.0.4
  Downloaded unicode-segmentation v1.7.0
  Downloaded unicode-xid v0.2.1
  Downloaded version_check v0.9.2
  Downloaded proc-macro-error-attr v1.0.4
   Compiling proc-macro2 v1.0.24
   Compiling version_check v0.9.2
   Compiling unicode-xid v0.2.1
   Compiling syn v1.0.48
   Compiling libc v0.2.80
   Compiling bitflags v1.2.1
   Compiling unicode-width v0.1.8
   Compiling unicode-segmentation v1.7.0
   Compiling vec_map v0.8.2
   Compiling strsim v0.8.0
   Compil

In [17]:
! cargo run this

    Finished dev [unoptimized + debuginfo] target(s) in 0.00s
     Running `target/debug/content this`
Hello, world!


# いまここ